# Projeto 2: Contando palavras por meio de PySpark: Espumante

## Grupo

| Nome | RA |
|---|---|
| Edgard Ramos | RA00320141 |
| Felipe Piovan | RA00319594 |
| Gabriel Galvão | RA00320082 |
| Manoela Finotti | RA00319067 |
| Melissa Assis | RA00320501 |
| Paulo Furtado | RA00274417 |

## Importando as bibliotecas

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=237aee85392eb55670ef70016be95d21f2c495c32f38c2b1c15021a125c9eb6b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
import requests
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pandas as pd
import re
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Criando a seção Spark

In [ ]:
spark = SparkSession.builder.appName("Espumante").getOrCreate()
sc = spark.sparkContext

## Funções Utilizadas

In [ ]:
def toDF(result):
    lista=[]
    for linha in result:
        dict_aux = {}
        dict_aux.update({"palavra": linha[0], "qtde": linha[1]})
        lista.append(dict_aux)
    df = pd.DataFrame(lista)
    return df

In [ ]:
def limpaHTML(pagina):
  soup = bs(pagina, 'html.parser')
  parag = soup.find_all('p')
  link=soup.find_all('a')
  linhas=[]
  for i in range(len(parag)):
    linhas.append(parag[i].text.lower())
  for i in range(len(link)):
    linhas.append(link[i].text.lower())
  return linhas

In [ ]:
def func(w,itens):
    result=""
    for i in itens:
        if i in w:
            result=i
            break
    return result

In [ ]:
def remove_stopwords(line):
    words = line.split()
    filtered_words = [word for word in words if word.lower() not in stop]
    return " ".join(filtered_words)


In [ ]:
def remove_punctuation(line):
    return re.sub(r'[^\w\s]', '', line)

In [ ]:
def remove_numbers(line):
    return re.sub(r'\d+', '', line)

In [ ]:
def count_items(input_file, items_list, phrase=False):
    if phrase:
        return input_file.map(remove_stopwords) \
                        .map(remove_punctuation) \
                        .map(remove_numbers) \
                        .map(lambda line: (func(line, items_list), 1) if func(line, items_list) != "" else (line, 0)) \
                        .reduceByKey(lambda a, b: a + b)
    else:
        return input_file.map(remove_stopwords) \
                        .map(remove_punctuation) \
                        .map(remove_numbers) \
                        .flatMap(lambda line: line.split()) \
                        .map(lambda word: (func(word, items_list), 1) if func(word, items_list) != "" else (word, 0)) \
                        .reduceByKey(lambda a, b: a + b)

In [ ]:
stop = stopwords.words('portuguese')
print(stop)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [ ]:
itens_single=["espumantes",	"champagne",	"adega",	"prosecco",	"casamento",	"casamentos",	"uvas",	"uva",	"vinícula",	"diversão",	"bebida",	"festas",	"festa",	"jantar",	"jantares",	"gastronomia",	"degustação",	"queijo",	"chocolate",	"chocolates",	"vinho",	"vinhos",	"enoturismo",	"vinhedo",	"taças",	"taça",	"restaurante",	"restaurantes",	"brinde",	"drink",	"sabor ",	"culinária",	"massas",	"bruschettas",	"frança",	"itália",	"espanha",	"anonovo",	"reveillon",	"frescor",	"sofisticação",	"cítricas",	"cítrico",	"refrescante",	"comidas ",	"aperitivos",	"sobremesas",	"frutado",	"chardonnay ",	"flores",	"joias",	"presentes",	"presente",	"cestas",	"peixe",	"peixes",	"marisco",	"mariscos",	"acidez",	"cerimonialista",	"decoração",	"fotografia",	"música",	"bartenders",	"bar",	"bares",	"confeitaria",	"doce",	"doces",	"arranjos",	"frisante",	"rótulos ",	"salton",	"chandon",	"freixenet",	"garibaldi ",	"aurora",	"miolo",	"noiva",	"noivas",	"safra", "brunch"]
itens_multiple = ["vinho espumante",	"happy hour",	"espumante nature",	"espumante extrabrut",	"espumante brut",	"espumante sec",	"espumante seco",	"espumante demisec",	"espumante doce",	"sabor frutado",	"frutos do mar",	"vinho frisante",	"ano novo",	"cave geisse",	"casa perini",	"casa valduga",	"vinícola aurora"]

## Loop

In [ ]:
sites = {
    "Revista Adega": [
        "https://revistaadega.uol.com.br/artigo/champagne-conheca-a-historia-da-criacao-dos-espumantes-e-brinde-as-mulheres-em-sua-vida.html",
        "https://revistaadega.uol.com.br/artigo/o-espumante-de-veneto-na-italia_3876.html",
        "https://revistaadega.uol.com.br/artigo/rei-charles-iii-aprova-selo-real-para-bebidas-incluindo-champagne.html",
        "https://revistaadega.uol.com.br/artigo/portugal-faz-espumante-ora-pois-conheca-o-melhor-vinho-espumante-da-terrinha.html",
        "https://revistaadega.uol.com.br/artigo/do-cava-tera-reserva-de-vinho-base-para-enfrentar-tempos-de-seca.html"
    ],
    "Dicas de Viagem": [
        "https://www.dicasdeviagem.com/onde-passar-o-reveillon-em-sao-paulo/",
        "https://www.dicasdeviagem.com/hoteis-romanticos-em-buenos-aires/",
        "https://www.dicasdeviagem.com/pontos-turisticos-de-paris/",
        "https://www.dicasdeviagem.com/hoteis-no-le-marais-em-paris/",
        "https://www.dicasdeviagem.com/chales-romanticos-em-sao-paulo/"
    ],
    "Rekparking": [
        "https://rekparking.com.br/garibaldi-e-as-6-rotas-da-capital-do-espumante/#:~:text=Atualmente%20%2C%20Garibaldi%20%C3%A9%20conhecida%20como,quanto%20pela%20qualidade%20dos%20produtos",
        "https://rekparking.com.br/4-coisas-que-voce-precisa-saber-sobre-enoturismo/",
        "https://rekparking.com.br/por-que-visitar-a-cidade-de-farroupilha/",
        "https://rekparking.com.br/bento-goncalves-e-5-rotas-imperdiveis/"
    ],
    "Vamos Receber": [
        "https://www.vamosreceber.com.br/2017/08/31/nossas-sugestoes-para-lista-de-casamento/",
        "https://www.vamosreceber.com.br/2018/03/07/questions-answers-qual-a-taca-certa/",
        "https://www.vamosreceber.com.br/2016/05/02/almoco-de-aniversario-azalea/",
        "https://www.vamosreceber.com.br/2017/10/09/um-jantar-em-familia-ou-alegria-em-tons-suaves/",
        "https://www.vamosreceber.com.br/2018/07/23/bar-latino/"
    ],
    "Porto Alegre Secreta": [
        "https://portoalegresecreta.com/melhores-restaurantes-porto-alegre/",
        "https://portoalegresecreta.com/candlelight-porto-alegre/",
        "https://portoalegresecreta.com/dining-in-the-dark-porto-alegre/",
        "https://portoalegresecreta.com/gatzz-gramado/"
    ],
    "Guia da Semana": [
         "https://www.guiadasemana.com.br/restaurantes/galeria/restaurantes-dia-maes-sao-paulo-2024",
         "https://www.guiadasemana.com.br/turismo/galeria/conheca-cabana-amalfi-serra-negra",
         "https://www.guiadasemana.com.br/restaurantes/noticia/happy-hour-com-espumante",
         "https://www.guiadasemana.com.br/receitas/galeria/drinks-com-espumante-para-as-festas-de-fim-de-ano",
         "https://www.guiadasemana.com.br/na-cidade/noticia/festas-de-reveillon-em-sao-paulo-2019"
    ],
    "Elite Vinho": [
        "https://elitevinho.com.br/blog/historia-e-origem-do-espumante-quem-criou/",
        "https://elitevinho.com.br/blog/alcool-e-espumante-tudo-o-que-voce-precisa-saber/",
        "https://elitevinho.com.br/blog/bodegas-chandon-espumantes-que-celebram-a-vida/",
        "https://elitevinho.com.br/blog/espumante-para-casamentos-como-escolher-o-ideal/"
    ],
    "Exame": [
         "https://exame.com/casual/os-vinhos-e-espumantes-com-maior-risco-de-ressaca-segundo-a-proteste/",
         "https://exame.com/casual/os-melhores-vinhos-tintos-do-chile-eleitos-por-um-dos-maiores-guias-do-mundo/",
         "https://exame.com/casual/dia-dos-namorados-restaurantes-para-comemorar-em-sao-paulo-2024/",
         "https://exame.com/casual/melhores-espumantes-moscatel-brasil/",
         "https://exame.com/casual/melhores-espumantes-do-brasil-metodo-tradicional"
    ],
    "Clube do Barman": [
        "https://clubedobarman.com/reveillon/",
        "https://clubedobarman.com/cardapio-de-brunch/",
        "https://clubedobarman.com/brinde-perfeito/",
        "https://clubedobarman.com/brunch/"
    ],
    "Alta Busca": [
        "https://altabusca.com.br/melhor-perfume-feminino",
        "https://altabusca.com.br/melhor-espumante/",
        "https://altabusca.com.br/melhor-adega-climatizada",
        "https://altabusca.com.br/melhor-vinho-rose",
        "https://altabusca.com.br/melhor-vinho-branco"
    ],
    "Blog Famiglia Valduga": [
        "https://blog.famigliavalduga.com.br/conheca-as-diferencas-de-espumantes-e-suas-caracteristicas",
        "https://blog.famigliavalduga.com.br/espumantes-para-servir-nas-festas-de-reveillon",
        "https://blog.famigliavalduga.com.br/sugestoes-de-espumantes-para-voce-curtir-o-ano-novo-em-boa-companhia",
        "https://blog.famigliavalduga.com.br/presenteie-quem-voce-ama-com-os-nossos-kits-de-natal",
        "https://blog.famigliavalduga.com.br/rotulos-para-o-alto-verao"
    ],
    "Oh My Gold": [
        "https://www.ohmygold.com.br/blog-tendencias-semijoias-casamento-2023",
        "https://www.ohmygold.com.br/blog/acessorios-para-reveillon",
        "https://www.ohmygold.com.br/tendencias-2024",
        "https://www.ohmygold.com.br/colecao-pedras-fusion/brincos-com-pedras-fusion",
        "https://www.ohmygold.com.br/blog/colares-da-moda-valorize-o-look-com-um-mix-de-colares"
    ],
    "Enologia": [
        "https://www.enologia.org.br/curiosidade/historia-do-champagne",
        "https://www.enologia.org.br/noticia/2017-e-o-ano-do-concurso-do-espumante-brasileiro",
        "https://www.enologia.org.br/noticia/concurso-do-espumante-brasileiro-mais-amostras-mais-vinicolas",
        "https://www.enologia.org.br/noticia/espumantes-brasileiros-atravessam-atlantico-e-conquistam-premios-na-hungria",
        "https://www.enologia.org.br/noticia/concurso-do-espumante-brasileiro-avancando-pelo-brasil"
    ],
    "Santa Augusta": [
        "https://santaaugusta.com.br/aprenda-a-utilizar-espumantes-em-receitas-de-drinks",
        "https://santaaugusta.com.br/sobre-nos",
        "https://santaaugusta.com.br/descubra-os-segredos-da-harmonizacao-de-ostras-e-espumantes",
        "https://santaaugusta.com.br/aprenda-a-tecnica-da-sabrage-para-abrir-espumantes",
        "https://santaaugusta.com.br/vinhos-e-espumantes-tipos-de-tacas-e-suas-funcoes"
    ],
    "Garibaldi Blog": [
        "https://www.garibaldiblog.com.br/post/gelarespumante",
        "https://www.garibaldiblog.com.br/post/carnaval-e-pedida-para-quebrar-paradigmas-na-hora-de-eleger-os-drinks-da-folia",
        "https://www.garibaldiblog.com.br/post/garibaldi-drinks-vinhos-e-espumantes",
        "https://www.garibaldiblog.com.br/post/relaxfrisante",
        "https://www.garibaldiblog.com.br/post/harmonizarespumante"
    ],
    "G1 Globo": [
        "https://g1.globo.com/economia/negocios/noticia/2023/12/28/por-que-os-espumantes-sao-a-bebida-favorita-das-festas-de-fim-de-ano.ghtml",
        "https://g1.globo.com/pr/parana/especial-publicitario/porto-a-porto/guia-do-vinho-e-da-gastronomia/noticia/2024/01/27/metodo-champenoise-x-metodo-charmat-qual-produz-o-melhor-espumante.ghtm",
        "https://g1.globo.com/pr/parana/especial-publicitario/porto-a-porto/guia-do-vinho-e-da-gastronomia/noticia/2024/06/08/nao-de-flores-de-vinho-rotulos-para-o-dia-dos-namorados.ghtml",
        "https://g1.globo.com/economia/agronegocios/agro-de-gente-pra-gente/noticia/2023/08/02/espumante-surge-de-vinho-tranquilo-e-bolhas-sao-formadas-com-a-ajuda-de-fungos.ghtml",
        "https://g1.globo.com/rs/rio-grande-do-sul/noticia/2023/12/27/tim-tim-maior-produtor-do-pais-rs-espera-alta-de-ate-10percent-nas-vendas-dos-espumantes-neste-fim-de-ano.ghtml"
    ],
    "Gauchazh Clicrbs": [
        "https://gauchazh.clicrbs.com.br/colunistas/marta-sfredo/noticia/2022/07/vinicola-gaucha-aposta-em-produtos-premium-para-aumentar-as-vendas-cl628w89c001i014s0eq2ztxf.html",
        "https://gauchazh.clicrbs.com.br/pioneiro/colunistas/juliana-bevilaqua/noticia/2023/12/melhores-espumantes-artesanais-do-brasil-sao-da-serra-clq444tsr0032016x1a53enlm.html",
        "https://gauchazh.clicrbs.com.br/destemperados/experiencias/serra-gaucha/noticia/2021/12/vinhos-e-alta-gastronomia-roteiro-para-um-bate-volta-em-flores-da-cunha-ckxah5nyf0034015pgy5hiyyf.html",
        "https://gauchazh.clicrbs.com.br/pioneiro/economia/noticia/2022/12/conquista-da-d-o-altos-de-pinto-bandeira-na-serra-valoriza-a-producao-de-espumantes-brasileiros-no-mundo-clb6wfe2a00ar014u8dqv9ccs.html",
        "https://gauchazh.clicrbs.com.br/pioneiro/economia/noticia/2023/07/conheca-os-espumantes-da-serra-premiados-com-medalha-de-ouro-na-franca-cljx14qzi005c015l3pwp58n6.html"
    ],
    "Evino Blog": [
        "https://www.evino.com.br/blog/caminho-do-vinho/",
        "https://www.evino.com.br/blog/champanhes-e-espumantes-ano-novo/",
        "https://www.evino.com.br/blog/espumante-o-que-e-descubra-o-universo-por-tras-das-bolhas/",
        "https://www.evino.com.br/blog/tabua-de-frios/",
        "https://www.evino.com.br/blog/prosecco-descubra-o-que-e-caracteristicas-e-curiosidades"
    ],
     "Blog Monte Carlo": [
        "https://blog.montecarlo.com.br/guia-de-presentes/joias-com-significado-para-celebrar-o-amor",
        "https://blog.montecarlo.com.br/guia-de-presentes/surpreenda-seu-amor-na-data-mais-romantica-do-ano",
        "https://blog.montecarlo.com.br/noivas/maio-mes-especial-noivas",
        "https://blog.montecarlo.com.br/noivas/o-pedido-perfeito-comeca-com-a-alianca-ideal",
        "https://blog.montecarlo.com.br/lancamentos/o-amor-esta-no-ar-lancamento-elo "
     ],
    "Mixology News": [
        "https://mixologynews.com.br/receitas/saint-turim-ana-paula-ulrich/",
        "https://mixologynews.com.br/receitas/saint-turim-ana-paula-ulrich/ ",
        "https://mixologynews.com.br/receitas/galego-frances-de-vinicius-laurentino/",
        "https://mixologynews.com.br/receitas/drinks-hidromel/",
        "https://mixologynews.com.br/receitas/bellini-receita/"
    ]
}

## Visualizando a Quantidade de Palavras de Cada Site

In [ ]:
# Função para processar os links de um site específico
def processar_site(site):
    if site not in sites:
        print(f"Site {site} não encontrado.")
        return None

    links = sites[site]
    dados = []

    for page in tqdm(links, desc=f"Processando {site}"):
        response = requests.get(page)
        linhas = limpaHTML(response.text)
        dados += linhas

    input_file = sc.parallelize(dados, 3)

    counts_single = count_items(input_file, itens_single, phrase=False)
    counts_multiple = count_items(input_file, itens_multiple, phrase=True)

    result_s = counts_single.collect()
    result_m = counts_multiple.collect()
    result = result_s + result_m

    df = toDF(result)
    if hasattr(df, 'qtde'):
        dfaux = df.query("qtde > 0")
        dfaux_sorted = dfaux.sort_values(by="qtde", ascending=False)
        return dfaux_sorted
    else:
        print("Sem palavras da lista.")
        return None

    return dfaux_sorted

# Função para processar todos os sites e armazenar os DataFrames em um dicionário
def processar_todos_os_sites():
    dfs_por_site = {}

    for site in sites:
        df = processar_site(site)
        if df is not None:
            dfs_por_site[site] = df

    return dfs_por_site

In [ ]:
# Exemplo de como processar e acessar o DataFrame de um site específico
site_especifico = "Mixology News"
df_site_especifico = processar_site(site_especifico)

if df_site_especifico is not None:
    print(f"DataFrame para o site {site_especifico}:")
    print(df_site_especifico)
else:
    print(f"Não foi possível processar o site {site_especifico}.")

Processando Mixology News: 100%|██████████| 5/5 [00:12<00:00,  2.58s/it]


DataFrame para o site Mixology News:
             palavra  qtde
821              bar   129
429           bebida    23
552            drink    19
21             vinho    12
827           itália     9
133          cítrico     6
1079            taça     6
135          frutado     5
1222        ano novo     5
221      restaurante     3
430       espumantes     3
48              uvas     3
852          frescor     2
876          chandon     2
867           flores     2
1351  espumante brut     2
848         cítricas     2
451           acidez     2
352        champagne     2
1070             uva     1
1140        prosecco     1
330            taças     1
743       bartenders     1


In [ ]:
# Exemplo de como processar e visualizar todos os DataFrames
dfs_todos_os_sites = processar_todos_os_sites()

for site, df in dfs_todos_os_sites.items():
    print(f"DataFrame para o site {site}:")
    print(df)
    print('\n')

Processando Mixology News: 100%|██████████| 5/5 [00:12<00:00,  2.47s/it]


DataFrame para o site Revista Adega:
              palavra  qtde
9               vinho   166
2           champagne    30
911          prosecco    23
883               uva    19
560        espumantes    18
27               uvas    16
524           vinhedo    16
284           espanha    13
912            itália     9
84              safra     8
1086       enoturismo     6
120            frança     4
543            bebida     4
73              adega     4
1116          chandon     3
147             taças     3
800               bar     3
1032          frescor     3
1035             taça     2
1196  vinho espumante     2
897            flores     2
843              doce     2
233           frutado     2
941         casamento     1
621            acidez     1
482      sofisticação     1
0              brinde     1


DataFrame para o site Dicas de Viagem:
           palavra  qtde
1933           bar    88
105    restaurante    27
2796      ano novo    15
498         frança    11
390      deco

In [ ]:
# Salvar os DataFrames em um arquivo Excel com abas separadas
with pd.ExcelWriter("sites.xlsx") as writer:
    for site, df in dfs_todos_os_sites.items():
        nome_aba = site
        df.to_excel(writer, sheet_name=nome_aba, index=False)
        print(f"Resultados para o site {site} foram salvos na aba {nome_aba}")

Resultados para o site Revista Adega foram salvos na aba Revista Adega
Resultados para o site Dicas de Viagem foram salvos na aba Dicas de Viagem
Resultados para o site Rekparking foram salvos na aba Rekparking
Resultados para o site Vamos Receber foram salvos na aba Vamos Receber
Resultados para o site Porto Alegre Secreta foram salvos na aba Porto Alegre Secreta
Resultados para o site Guia da Semana foram salvos na aba Guia da Semana
Resultados para o site Elite Vinho foram salvos na aba Elite Vinho
Resultados para o site Exame foram salvos na aba Exame
Resultados para o site Clube do Barman foram salvos na aba Clube do Barman
Resultados para o site Alta Busca foram salvos na aba Alta Busca
Resultados para o site Blog Famiglia Valduga foram salvos na aba Blog Famiglia Valduga
Resultados para o site Oh My Gold foram salvos na aba Oh My Gold
Resultados para o site Enologia foram salvos na aba Enologia
Resultados para o site Santa Augusta foram salvos na aba Santa Augusta
Resultados par